https://github.com/DeVyacheslav/PyTorch-MNIST-DCGAN/blob/master/MNIST_DCGAN.ipynb

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import torchvision.utils as vutils

In [1]:
class Discriminator(nn.Module):
  def __init__(self, channels_img, features_d):
    super(Discriminator, self).__init__()

    self.net = nn.Sequential(
        # N x channels_img x 64 x 64
        nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(features_d),
        nn.LeakyReLU(0.2),

        # N x features_d x 32 x 32
        nn.Conv2d(features_d, features_d*2, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(features_d*2),
        nn.LeakyReLU(0.2),

        # N x features_d*2 x 16 x 16
        nn.Conv2d(features_d*2, features_d*4, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(features_d*4),
        nn.LeakyReLU(0.2),

        # N x features_d*4 x 8 x 8
        nn.Conv2d(features_d*4, features_d*8, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(features_d*8),
        nn.LeakyReLU(0.2),

        # N x features_d*8 x 4 x 4
        nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0),
    )

  def forward(self, x):
    return self.net(x) 

In [2]:
class Generator(nn.Module):
  def __init__(self, channels_noise, channels_img, features_g):
    super(Generator, self).__init__()

    self.net = nn.Sequential(
        # N x channels_noise x 1 x 1
        nn.ConvTranspose2d(channels_noise, features_g*16, kernel_size=4, stride=1, padding=0),
        nn.BatchNorm2d(features_g*16),
        nn.ReLU(),

        # N x features_g*16 x 4 x 4
        nn.ConvTranspose2d(features_g*16, features_g*8, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(features_g*8),
        nn.ReLU(),

        # N x features*8 x 8 x 8
        nn.ConvTranspose2d(features_g*8, features_g*4, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(features_g*4),
        nn.ReLU(),

        # N x features*4 x 16 x 16
        nn.ConvTranspose2d(features_g*4, features_g*2, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(features_g*2),
        nn.ReLU(),

        # N x features*2 x 32 x 32
        nn.ConvTranspose2d(features_g*2, channels_img, kernel_size=4, stride=2, padding=1),
        nn.Tanh()
    )

  def forward(self, x):
    return self.net(x) 

In [ ]:
lr_D = 0.0002
lr_G = 0.0002
batch_size = 64
image_size = 64
channels_img = 3
channels_noise = 256
num_epochs = 100

features_d = 32
features_g = 32

In [ ]:
custom_transforms = transforms.Compose([
  transforms.Resize(image_size),
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = datasets.CIFAR10(root='dataset/', train=True, transform=custom_transforms, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
  
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
net_D = Discriminator(channels_img, features_d).to(device)
net_G = Generator(channels_noise, channels_img, features_g).to(device)

optimizator_D = optim.Adam(net_D.parameters(), lr=lr_D)
optimizator_G = optim.Adam(net_G.parameters(), lr=lr_G)

net_D.train()
net_G.train()

criterion = nn.MSELoss()

real_label = 1
fake_label = 0

fixed_noise = torch.randn(64, channels_noise, 1, 1).to(device)

In [3]:
print('Start training')

for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(dataloader):
    ### Discriminator training
    data = data.to(device)
    batch_size = data.shape[0]

    net_D.zero_grad()
    labels = (torch.ones(batch_size)*0.9).to(device)

    output = net_D(data).reshape(-1)
    loss_D_real = criterion(output, labels)

    D_mean = output.mean().item()

    noise = torch.randn(batch_size, channels_noise, 1, 1).to(device)
    fake = net_G(noise) 
    labels = (torch.ones(batch_size)*0.1).to(device)

    output = net_D(fake.detach()).reshape(-1)
    loss_D_fake = criterion(output, labels)

    loss_D = loss_D_real + loss_D_fake
    loss_D.backward()

    optimizator_D.step()
    
    ### Generator training
    net_G.zero_grad()
    labels = torch.ones(batch_size).to(device)

    output = net_D(fake).reshape(-1)
    loss_G = criterion(output, labels)
    loss_G.backward()
    optimizator_G.step()
    
  fake = net_G(fixed_noise)
  vutils.save_image(fake.detach(),'output/fake_samples_epoch_%03d.png' %(epoch),normalize=True)
    
    
  print(f'Epoch {epoch}/{num_epochs}, Batch {batch_idx}/{len(dataloader)} \
  Loss D {loss_D:.4f}, Loss G: {loss_G:.4f}, D_mean: {D_mean:.4f}')


Files already downloaded and verified
Start training
Epoch 0/100, Batch 781/782   Loss D 0.0527, Loss G: 0.7492, D_mean: 0.7156
Epoch 1/100, Batch 781/782   Loss D 0.0744, Loss G: 0.6612, D_mean: 0.8415
Epoch 2/100, Batch 781/782   Loss D 0.0531, Loss G: 0.5952, D_mean: 0.7488
Epoch 3/100, Batch 781/782   Loss D 0.0346, Loss G: 1.0665, D_mean: 0.8403
Epoch 4/100, Batch 781/782   Loss D 0.0236, Loss G: 0.8746, D_mean: 0.8157
Epoch 5/100, Batch 781/782   Loss D 0.0073, Loss G: 0.8400, D_mean: 0.9308
Epoch 6/100, Batch 781/782   Loss D 0.0666, Loss G: 0.5909, D_mean: 0.7573
Epoch 7/100, Batch 781/782   Loss D 0.1072, Loss G: 0.6963, D_mean: 0.6719
Epoch 8/100, Batch 781/782   Loss D 0.0616, Loss G: 0.9452, D_mean: 0.8488
Epoch 9/100, Batch 781/782   Loss D 0.0735, Loss G: 0.7592, D_mean: 0.7215
Epoch 10/100, Batch 781/782   Loss D 0.0833, Loss G: 0.6415, D_mean: 0.8255
Epoch 11/100, Batch 781/782   Loss D 0.2369, Loss G: 0.8108, D_mean: 0.4955
Epoch 12/100, Batch 781/782   Loss D 0.0986, 